In [1]:
import math 
from collections import Counter
from graphviz import Digraph
import pydotplus
import numpy as np
import pandas as pd
import random
import os
os.system("cls")
import json

In [2]:
class DataProcessor:
    def __init__(self, file_name):
        self.file_name = file_name
        self.data = pd.read_csv(file_name).dropna().drop(['nameOrig','nameDest'],axis=1)

        self.min_max_values = self.calculate_min_max()
        self.train=[]
        self.test=[] 
        self.label_test=[]
        self.label_train=[]
        self.temp=self.calculate_min_max()
    

    def calculate_min_max(self):
        return {
            'amount': (self.data['amount'].min(), self.data['amount'].max()),
            'oldbalanceOrg': (self.data['oldbalanceOrg'].min(), self.data['oldbalanceOrg'].max()),
            'newbalanceOrig': (self.data['newbalanceOrig'].min(), self.data['newbalanceOrig'].max()),  
            'oldbalanceDest': (self.data['oldbalanceDest'].min(), self.data['oldbalanceDest'].max()),
            'newbalanceDest': (self.data['newbalanceDest'].min(), self.data['newbalanceDest'].max()),  
        }

    def discretize(self, value, min_val, max_val, length):
        return np.floor((value - min_val) / ((max_val - min_val) / length))

    def edit_row(self, row):
        label = row.pop('isFraud')

        name = 'type'
        type_mapping = {
                'PAYMENT': 0,
                'TRANSFER': 1,
                'CASH_OUT': 2,
                'DEBIT': 3,
                'CASH_IN': 4
            }
        if row[name] in type_mapping:
            row[name] = type_mapping[row[name]]
        else:
            row[name] = type_mapping['CASH_IN'] 

        for name in ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']:
            min_val, max_val = self.min_max_values[name]
            row[name] = self.discretize(row[name], min_val, max_val, 8 if name != 'amount' else 6)
    
        return row, int(label)

    def read_csv(self):
        pos_list = self.data[self.data['isFraud'] == 1].sample(frac=1).to_dict('records')
        neg_list = self.data[self.data['isFraud'] == 0].sample(frac=1).to_dict('records')

        selected_data = pos_list[:211] + neg_list[:9788]
        selected_test = pos_list[-211 + 100:-1] + neg_list[-9788 + 100:-1]


        return selected_data, selected_test

    def process_data(self):
        selected_data, selected_test = self.read_csv()
      
        for row in selected_data:
            xt1,yt1=self.edit_row(row) 
            self.train.append(xt1)
            self.label_train.append(yt1)
        for row in selected_test:
            xt1,yt1=self.edit_row(row) 
            self.test.append(xt1)
            self.label_test.append(yt1)
    

        return 


def unique_values(data):
    result = {}
    for dictionary in data:
        for key, value in dictionary.items():
            if key not in result:
                result[key] = set()
            result[key].add(value)
    return result


In [3]:
class Node:
    def __init__(self, attribute=None, value=None, children=None, label=None):
        self.attribute = attribute
        self.value = value
        self.children = children if children else {}
        self.label = label
class DecisionTree:
    def __init__(self):
        self.Node = Node

    def calculate_entropy(self, data):
        num_data = len(data)
        if num_data == 0:
            return 0

        num_positives = sum(label for _, label in data)
        num_negatives = num_data - num_positives

        if num_positives == 0 or num_negatives == 0:
            return 0

        p_positive = num_positives / num_data
        p_negative = num_negatives / num_data

        return -p_positive * math.log2(p_positive) - p_negative * math.log2(p_negative)

    def calculate_gini_index(self, data):
        num_data = len(data)
        if num_data == 0:
            return 0

        num_positives = sum(label for _, label in data)
        num_negatives = num_data - num_positives

        p_positive = num_positives / num_data
        p_negative = num_negatives / num_data

        return 1 - (p_positive ** 2 + p_negative ** 2)

    def calculate_information_gain(self, data, attribute, criterion, unique_values_data):
        if criterion == 'gini':
            impurity_func = self.calculate_gini_index
        elif criterion == 'entropy':
            impurity_func = self.calculate_entropy
        else:
            raise ValueError("Invalid criterion")

        num_data = len(data)
        impurity_after = 0.0

        values = set(unique_values_data[attribute])
        for value in values:
            subset = [(row, label) for row, label in data if row[attribute] == value]
            weight = len(subset) / num_data
            impurity_after += weight * impurity_func(subset)

        impurity_before = impurity_func(data)
        information_gain = impurity_before - impurity_after
        return information_gain

    def select_best_attribute(self, data, attributes, criterion, unique_values_data):
        best_info_gain = -1
        best_attribute = None

        for attribute in attributes:
            info_gain = self.calculate_information_gain(data, attribute, criterion, unique_values_data)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_attribute = attribute

        return best_attribute

    def most_common_element(self, labels):
        label_counts = Counter(labels)
        most_common_label = label_counts.most_common(1)
        return most_common_label[0][0]

    def Build_tree(self, training_data, attributes, mode):
        labels = [label for _, label in training_data]
        x_datas=[x_data for x_data,_ in training_data]

        if labels.count(labels[0]) == len(labels): return self.Node(label=labels[0])
        if not attributes: return self.Node(label=self.most_common_element(labels))

        best_attr = self.select_best_attribute(training_data, attributes, mode, unique_values_data)
        tree = self.Node(attribute=best_attr)
        attributes.remove(best_attr)

        values = list(unique_values_data[best_attr])
        for value in values:
            subset = []
            for feature_row, label in training_data:
                if feature_row[best_attr] == value:
                    subset.append((feature_row, label))

            if not subset:
                tree.children[value] = self.Node(label=self.most_common_element(labels))
            else:
                tree.children[value] = self.Build_tree(subset, attributes.copy(), mode)

        return tree

    def predict(self, node, x_data):
        if node.label is not None:
            return node.label
        elif node.attribute is None:
            return self.most_common_element(data_class.label_train)
        else:
            value = x_data[node.attribute]
            child_node = node.children.get(value)
            if child_node:
                return self.predict(child_node, x_data)
            else:
                return self.most_common_element(data_class.label_train)

    def calculate_accuracy(self, tree, test_data):
        correct = 0
        for x_data, y_label in test_data:
            prediction = self.predict(tree, x_data)
            if prediction == y_label:
                correct += 1
        accuracy = correct / len(test_data)
        return accuracy

In [4]:
dt = DecisionTree()
file_name = 'onlinefraud.csv'
data_class= DataProcessor(file_name)
data_class.process_data()

train=data_class.train
label=data_class.label_train
attributes = list(train[0].keys())
training_data = [(train[i], label[i]) for i in range(len(train))]
unique_values_data=unique_values(train)

EntropyTree     = dt.Build_tree(training_data, attributes, mode='entropy')
GiniTree = dt.Build_tree(training_data, attributes, mode='gini')
test = data_class.test
test_label = data_class.label_test
test_data = [(test[i], test_label[i]) for i in range(len(test_label))]
EntropyAccuracy = dt.calculate_accuracy(EntropyTree, test_data)
print("Entropy:", EntropyAccuracy*100)

GiniAccuracy = dt.calculate_accuracy(GiniTree, test_data)
print("Gini:", GiniAccuracy*100)

Entropy: 98.80575686434622
Gini: 98.8261712769215


In [5]:
from graphviz import Digraph

def visualize_tree(node, dot=None, parent_name=None, edge_label='', node_style=None, edge_style=None):
    if dot is None:
        dot = Digraph(comment='Decision Tree')
    if node_style is None:
        node_style = {'shape': 'box', 'style': 'filled', 'fillcolor': 'lightblue'}
    if edge_style is None:
        edge_style = {'fontsize': '11', 'fontcolor': 'red'}
    node_name = f'node_{id(node)}'

    if hasattr(node, 'label') and node.label is not None:
        dot.node(node_name, str(node.label), **node_style)
        if parent_name:
            dot.edge(parent_name, node_name, label=edge_label, **edge_style)
    else:
        internal_node_style = {'shape': 'ellipse', 'style': 'filled', 'fillcolor': 'lightyellow'}
        dot.node(node_name, str(node.attribute), **internal_node_style)
        if parent_name:
            dot.edge(parent_name, node_name, label=edge_label, **edge_style)
        for value, child_node in node.children.items():
            dot = visualize_tree(child_node, dot, node_name, str(value), node_style, edge_style)
    return dot
node_style = {'shape': 'box', 'style': 'filled', 'fillcolor': 'lightgreen'}
edge_style = {'fontsize': '11', 'fontcolor': 'blue'}
dot = visualize_tree(EntropyTree, node_style=node_style, edge_style=edge_style)
dot.render('entropy_tree', format='pdf', view=True)

dot = visualize_tree(GiniTree, node_style=node_style, edge_style=edge_style)
dot.render('gini_tree', format='pdf', view=True)


'gini_tree.pdf'